# StackOverflow answer classifier

In [6]:
import re
import os
from html.parser import HTMLParser
from xml.etree import ElementTree as etree
from xml.etree.ElementTree import Element

import numpy as np
import pandas as pd

In [19]:
class HTML2String(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.strict = False
        self.convert_charrefs= True
        self.text = []
    def handle_data(self, d):
        self.text.append(d)
    def get_data(self):
        return ''.join(self.text)

def html_to_string(html):
    s = HTML2String()
    s.feed(html)
    return s.get_data()

def code_line_count(html):
    count_lines = 0
    codes = re.findall(r'<code>.*?</code>', html, re.DOTALL)
    for code in codes:
        text = html_to_string(code)
        lines = text.split('\n')
        lines = list(filter(lambda x: len(x) > 0, lines))
        count_lines += len(lines)
    return count_lines

In [14]:
dict(elem.items())

{'AnswerCount': '1',
 'Body': '<p>I\'m trying to set up an app that configures my instances upon launch and I want to close down that app\'s API access as much as possible. My current policy is as follows:</p>\n\n<pre><code>{\n  "Version": "2012-10-17",\n  "Statement": [\n    {\n      "Sid": "Stmt1388183890000",\n      "Effect": "Allow",\n      "Action": [\n        "ec2:AssociateAddress",\n        "ec2:CreateTags",\n        "ec2:DescribeInstances",\n        "ec2:RebootInstances"\n      ],\n      "Resource":"*"\n    }\n  ]\n}\n</code></pre>\n\n<p>However, this allows the app to perform any of these actions on anything in EC2. Is there a way I can lock down the actions of the app on an ec2 instance to either that specific instance, or to all of the boxes that have the same IAM role?</p>\n',
 'CommentCount': '0',
 'CreationDate': '2014-01-01T00:03:39.953',
 'Id': '20864429',
 'LastActivityDate': '2014-01-01T07:28:22.857',
 'OwnerUserId': '382775',
 'PostTypeId': '1',
 'Score': '0',
 'Tags

In [ ]:
def question_to_df(question):
    q_id = question['Id']
    author_id = question['OwnerUserId']
    date = question['CreationDate']
    title = question['Id']
    text = html_to_string(question['Id'])
    score = question['Id']
    view_count = question.['Id']
    comment_count = question['Id']
    tags = question['Id']
    url = question['Id']

    info = [q_id, author_id, date, title, text, score, view_count, answer_count, comment_count, tags, url]
    q_df = pd.DataFrame([info], columns=qs_columns)
    return q_df

def answer_to_df(answer):
    print('b')
    a_id = answer.id
    author_id = answer.owner_id
    question_id = answer.question_id
    date = answer.creation_date
    text = html_to_string(answer.body)
    code_lines = code_line_count(answer.body)
    score = answer.score
    accepted = answer.accepted
    url = answer.url

    info = [a_id, author_id, question_id, date, text, code_lines, score, accepted, url]
    a_df = pd.DataFrame([info], columns=as_columns)
    return a_df

def user_to_df(user):
    print('c')
    u_id = user.id
    name = user.display_name
    total_questions = so.questions(user_id=user.id).total
    unanswered_questions = count_unanswered_questions(user)
    total_answers = so.answers(user_id=user.id).total
    accepted_answers = count_accepted_answers(user)
    reputation = user.reputation.real
    badge_total = user.badge_total
    url = user.url

    info = [u_id, name, total_questions, unanswered_questions, total_answers, accepted_answers, reputation, badge_total, url]
    u_df = pd.DataFrame([info], columns=us_columns)
    return u_df

In [7]:
qs_columns = ['id', 'author_id', 'date', 'title', 'text', 'score', 'view_count', 'answer_count', 'comment_count', 'tags', 'url']
as_columns = ['id', 'author_id', 'question_id', 'date', 'text', 'code_lines', 'score', 'accepted', 'url']
us_columns = ['id', 'name', 'total_questions', 'unanswered_questions', 'total_answers', 'accepted_answers', 'reputation', 'badge_total', 'url']
qa_columns = ['id', 'text', 'q_score', 'a_score', 'qer_reputation', 'aer_reputation', 'qer_percent_answered_questions', 'aer_percent_accepted_answers', 'aer_badge_total', 'a_code_lenth', 'is_qa']

questions_df = pd.DataFrame(columns=qs_columns)
answers_df = pd.DataFrame(columns=as_columns)
users_df = pd.DataFrame(columns=us_columns)
qas_df = pd.DataFrame(columns=qa_columns)

In [18]:
pd.DataFrame([[1,2,3]]).to_csv('2.csv')

In [9]:
data_path = '/media/antonio/92088d7f-1ed4-49dd-b55f-01462ab87ebb/so_data'

##Questions

In [10]:
xml_file = 'Questions-2014.xml'
xml_path = os.path.join(data_path, xml_file)

In [13]:
count = 0
iterparser = etree.iterparse(xml_path)
iterator = iter(iterparser)
event, root = next(iterator)
root.clear()

for event, elem in iterparser:
    print(dict(elem.items()))
    count +=1
    if count == 4:
        break
    elem.clear()

{'CreationDate': '2014-01-01T00:02:46.990', 'PostTypeId': '1', 'OwnerUserId': '1377324', 'AnswerCount': '1', 'Score': '0', 'Body': '<p>I\'ve been looked around and have no luck on this.</p>\n\n<p>So I have a complex query to sphinx and wondering if I could do complex query of SQL like this</p>\n\n<pre><code>SELECT * FROM Table WHERE (categoryid in (17,18,19, 20) OR merchantid in (110,112,335)) AND brandid NOT in (20, 30, 40,50);\n</code></pre>\n\n<p>I know OR is working with SetSelect with</p>\n\n<pre><code>$s-&gt;SetSelect(\'*, IN(categoryid, 17,18 19) OR IN(merchantid,110,112, 335) AS test\');\n$s-&gt;SetFilter(\'test\', array(1), true);\n$s-&gt;query("", "index");\n</code></pre>\n\n<p>NOT query could be done as individual query </p>\n\n<pre><code>$s-&gt;SetSelect(\'*, IN(brandid,20,30,40,50 AS nottest\');\n$s-&gt;SetFilter(\'nottest\', array(1));\n$s-&gt;query("", "index");\n</code></pre>\n\n<p>But how should I combine both in one query</p>\n', 'Id': '20864423', 'Tags': '<sphinx>', 

In [12]:
elem.get('Id')

'82956'

In [18]:
elem.items()

[('UserId', '652'),
 ('Id', '82956'),
 ('Date', '2008-09-15T08:55:03.957'),
 ('Name', 'Teacher')]

In [19]:
elem.get('Name')

'Teacher'

In [20]:
elem.tag

'row'

In [24]:
next(elem)

TypeError: 'xml.etree.ElementTree.Element' object is not an iterator

In [35]:
root.tag

'badges'

In [36]:
elem

<Element 'row' at 0x7f0098070368>

In [38]:
root.clear()

In [40]:
root.tag

'badges'

In [9]:
elem.items()

[('LastEditDate', '2014-01-23T22:52:51.697'),
 ('LastActivityDate', '2014-01-23T22:52:51.697'),
 ('LastEditorUserId', '3052751'),
 ('Id', '20864452'),
 ('Body',
  '<p>Is there a way to use textpath using SnapSVG? I tried using textPath as an attribute but it does not seem to add a textpath element in the text node.</p>\n\n<pre><code>var txtpth = s.path("M70 70 Q 80 90 200 150 L 200 400").attr({\n    fill: "none",\n    stroke: "black"\n });\nvar crooked = s.text(0,0,"lorempsum ipsum lorempsum ipsum lorempsum ipsum lorempsum   ipsum").attr({\n     textPath: txtpth,\n     stroke:"black"\n });\n</code></pre>\n\n<p>I do not see a direct API to manipulate text paths in SVG using snap.svg.</p>\n'),
 ('CreationDate', '2014-01-01T00:10:40.443'),
 ('AnswerCount', '3'),
 ('CommentCount', '2'),
 ('PostTypeId', '1'),
 ('Tags', '<javascript><text><svg><path><snap.svg>'),
 ('OwnerUserId', '2452938'),
 ('ViewCount', '1170'),
 ('Title', 'How to use textPath for text in snap.svg?'),
 ('Score', '1')]